* data 


Bibliotheque necessaire 

In [1]:
pip install googlesearch-python selenium webdriver-manager pillow opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U duckduckgo-search



  Attempting uninstall: duckduckgo-search
    Found existing installation: duckduckgo_search 7.5.4
    Uninstalling duckduckgo_search-7.5.4:
      Successfully uninstalled duckduckgo_search-7.5.4


In [3]:
pip install requests beautifulsoup4 trafilatura


In [4]:
pip install selenium pillow webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install selenium googlesearch-python webdriver-manager pillow


In [6]:
pip uninstall duckduckgo_search


^C
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install duckduckgo-search  # avec tiret, pas underscore


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [1]:
pip install undetected-chromedriver pillow duckduckgo-search


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install datasets


  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached pyarrow-19.0.1-cp313-cp313-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.11.14-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.2.0-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.3.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cac

DATA COLLECTION

In [ ]:
import os
import time
import random
import json
import csv
import pandas as pd
from tqdm import tqdm
from PIL import Image
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import requests

# === PARAMÈTRES GLOBAUX ===
NB_QUERIES = 7
NB_PAGES_PER_QUERY = 1000
MAX_LINKS = 1000000
WINDOW_WIDTH = 1280
MIN_HEIGHT = 800
MAX_HEIGHT = 10_000

IMG_DIR = "dataset_images"
ERROR_DIR = "errors"
META_JSON = "dataset_metadata.json"
META_CSV = "dataset_metadata.csv"
LOG_FILE = "log.txt"

SERPAPI_API_KEY = "edd29fc5094a91be55b8ee5fea56d43c2202cb7b858bb866284fa56dc5deccd0"  # 📅 Insère ta clé API ici

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(ERROR_DIR, exist_ok=True)
metadata = {}

# === Requêtes aléatoires ===
queries = [
    "AI OR Data Science OR Machine Learning OR Deep Learning",
    "education ",
    "blog OR politics OR news",
    "health OR fitness OR wellness",
    "Food OR Cooking OR Recipes",
    "fashion OR clothing OR style OR trends",
    "articles"
]

# === Recherche via SerpAPI ===
def get_google_links_serpapi(query, max_results=100):
    results = []
    page = 0
    while len(results) < max_results:
        url = "https://serpapi.com/search"
        params = {
            "engine": "google",
            "q": query,
            "api_key": SERPAPI_API_KEY,
            "num": 100,
            "start": page * 100
        }
        response = requests.get(url, params=params)
        data = response.json()
        if "error" in data:
            print(f"[ERREUR SERPAPI] {data['error']}")
            break

        organic = data.get("organic_results", [])
        for r in organic:
            link = r.get("link")
            if link:
                results.append(link)
            if len(results) >= max_results:
                break
        if not organic:
            break
        page += 1
    return results

# === Scroll automatique ===
def scroll_smooth(driver, pause_time=1.0, max_scrolls=30):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# === Démarrage du navigateur furtif ===
options = uc.ChromeOptions()
options.headless = True
options.add_argument(f"--window-size={WINDOW_WIDTH},{MIN_HEIGHT}")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(options=options)

# === Lecture JSON existant ===
if os.path.exists(META_JSON):
    with open(META_JSON, "r") as f:
        metadata = json.load(f)
        existing_urls = {v["url"] for v in metadata.values()}
else:
    existing_urls = set()

# === Collecte ===
with open(LOG_FILE, "w", encoding="utf-8") as log:
    total_count = len(metadata)
    selected_queries = random.sample(queries, NB_QUERIES)

    for qidx, query in enumerate(selected_queries):
        print(f"\n🔍 [{qidx+1}/{NB_QUERIES}] Requête : {query}")
        log.write(f"\n--- Query: {query} ---\n")

        links = get_google_links_serpapi(query, max_results=MAX_LINKS)
        print(f"[DEBUG] {len(links)} liens trouvés via SerpAPI pour : {query}")

        random.shuffle(links)
        success = 0

        for idx, url in tqdm(enumerate(links), total=NB_PAGES_PER_QUERY, desc=f"  → {query[:30]}..."):
            print(f"[DEBUG] → test de l'URL : {url}")
            if success >= NB_PAGES_PER_QUERY:
                break
            if url in existing_urls:
                continue

            try:
                driver.get(url)
                time.sleep(3)

                body = driver.find_element(By.TAG_NAME, "body").text.strip()
                if len(body) < 200:
                    print(f"[DEBUG] Ignorée (page trop vide) : {url}")
                    continue

                real_height = driver.execute_script(
                    "return Math.max(document.body.scrollHeight, document.documentElement.scrollHeight)"
                )
                target_height = max(MIN_HEIGHT, min(real_height, MAX_HEIGHT))
                driver.set_window_size(WINDOW_WIDTH, target_height)
                scroll_smooth(driver, pause_time=1, max_scrolls=20)
                driver.set_window_size(WINDOW_WIDTH, target_height)

                base_name = f"{query.replace(' ', '_')}_{success}_{int(time.time())}"
                temp_png = os.path.join(IMG_DIR, base_name + ".png")
                final_jpg = os.path.join(IMG_DIR, base_name + ".jpg")

                driver.save_screenshot(temp_png)
                img = Image.open(temp_png).convert("RGB")
                img.save(final_jpg, "JPEG", quality=90)
                print(f"[✓] Image capturée : {final_jpg}")
                os.remove(temp_png)

                metadata[base_name + ".jpg"] = {
                    "url": url,
                    "query": query,
                    "height": target_height
                }
                existing_urls.add(url)
                log.write(f"[✓] {base_name}.jpg → {url}\n")
                success += 1
                total_count += 1

            except Exception as e:
                print(f"[ERREUR] {url} → {e}")
                err_file = os.path.join(ERROR_DIR, f"error_{qidx}_{idx}.txt")
                with open(err_file, "w", encoding="utf-8") as f:
                    f.write(f"URL: {url}\nErreur: {str(e)}")
                log.write(f"[!] ERREUR {url}\n{str(e)}\n")
                continue
        print(f"✅ {success} images capturées pour la requête : {query}")

# === Sauvegarde JSON + CSV
with open(META_JSON, "w") as f:
    json.dump(metadata, f, indent=2)

csv_data = [
    {"filename": fname, "url": info["url"], "query": info["query"], "height": info["height"]}
    for fname, info in metadata.items()
]
df = pd.DataFrame(csv_data)
df.to_csv(META_CSV, index=False)

print(f"\n📅 {total_count} images capturées au total.")
print(f"📁 Images : {IMG_DIR}")
print(f"📄 Metadata JSON : {META_JSON}")
print(f"📄 Metadata CSV : {META_CSV}")
print(f"📝 Log : {LOG_FILE}")

driver.quit()


could not detect version_main.therefore, we are assuming it is chrome 108 or higher
100%|██████████| 3/3 [05:08<00:00, 102.73s/it]



✅ Annotation PRÉCISE terminée!
• Images traitées: 3/3
• Fichier COCO: annotations_precises.json
• Taille de référence: 1280x800


In [14]:
import os
import json
import time
from tqdm import tqdm
from PIL import Image, ImageDraw
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# Configuration
IMG_DIR = "dataset_images"
ANNOTATED_DIR = "dataset_images_annotated"
META_FILE = "dataset_metadata.json"
COCO_FILE = "annotations_corrected.json"
os.makedirs(ANNOTATED_DIR, exist_ok=True)

# Paramètres cohérents avec le script de capture
WINDOW_WIDTH = 1280
DRIVER_HEIGHT = 800  # Doit correspondre à MIN_HEIGHT du script de capture
TEST_LIMIT=5
# Classes et couleurs
CLASSES = {
    "title": "h1, h2",
    "content": "main, [role=main]",
    "media": "img, video",
    "header": "header",
    "footer": "footer",
    "ads": "[class*=ad], [id*=ad], iframe[src*='ads']",
    "sidebar": "[class*=sidebar]"
}

CLASS_COLORS = {
    "title": "red",
    "content": "blue",
    "media": "green",
    "header": "purple",
    "footer": "orange",
    "ads": "yellow",
    "sidebar": "cyan"
}

# Initialisation COCO
coco_output = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": i+1, "name": name, "supercategory": "web", "color": CLASS_COLORS[name]}
        for i, name in enumerate(CLASSES.keys())
    ]
}

# Initialisation Selenium
options = uc.ChromeOptions()
options.add_argument(f"--window-size={WINDOW_WIDTH},{DRIVER_HEIGHT}")
options.headless = True
driver = uc.Chrome(options=options)

# Chargement des métadonnées
with open(META_FILE) as f:
    metadata = json.load(f)

def get_element_position_with_scroll(driver, element):
    """Obtient la position absolue d'un élément en tenant compte du scroll"""
    return driver.execute_script(
        "var rect = arguments[0].getBoundingClientRect();"
        "return {"
        "  x: rect.left + window.pageXOffset,"
        "  y: rect.top + window.pageYOffset,"
        "  width: rect.width,"
        "  height: rect.height"
        "};",
        element
    )

# Processus d'annotation
for img_idx, (img_name, info) in enumerate(tqdm(list(metadata.items())[:TEST_LIMIT])):
    try:
        img_path = os.path.join(IMG_DIR, img_name)
        annotated_path = os.path.join(ANNOTATED_DIR, img_name)
        
        # Charger l'image originale
        original_img = Image.open(img_path)
        img_width, img_height = original_img.size
        
        # Configurer le navigateur
        driver.set_window_size(WINDOW_WIDTH, DRIVER_HEIGHT)
        driver.get(info["url"])
        time.sleep(3)
        
        # Pre-scroll pour charger tous les éléments
        driver.execute_script("window.scrollTo(0, 0)")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0)")
        time.sleep(1)
        
        # Créer une image annotée
        annotated_img = original_img.copy()
        draw = ImageDraw.Draw(annotated_img)
        
        # Enregistrer les infos de l'image
        coco_output["images"].append({
            "id": img_idx + 1,
            "file_name": img_name,
            "width": img_width,
            "height": img_height,
            "url": info["url"]
        })
        
        # Annoter chaque classe
        for class_id, (label, selector) in enumerate(CLASSES.items(), start=1):
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            color = CLASS_COLORS[label]
            
            for elem in elements:
                try:
                    # Obtenir la position absolue avec JavaScript
                    pos = get_element_position_with_scroll(driver, elem)
                    
                    # Convertir les coordonnées en fonction de la taille de l'image
                    x = int(pos['x'])
                    y = int(pos['y'])
                    w = int(pos['width'])
                    h = int(pos['height'])
                    
                    # Vérifier que l'élément est visible dans l'image
                    if (x + w < 0 or y + h < 0 or 
                        x > img_width or y > img_height or
                        w < 20 or h < 20):
                        continue
                    
                    # Ajuster les coordonnées si nécessaire
                    x = max(0, x)
                    y = max(0, y)
                    w = min(w, img_width - x)
                    h = min(h, img_height - y)
                    
                    # Dessiner l'annotation
                    draw.rectangle([x, y, x + w, y + h], outline=color, width=3)
                    draw.text((x + 5, y + 5), label, fill=color)
                    
                    # Ajouter à COCO
                    coco_output["annotations"].append({
                        "id": len(coco_output["annotations"]) + 1,
                        "image_id": img_idx + 1,
                        "category_id": class_id,
                        "bbox": [x, y, w, h],
                        "area": w * h,
                        "iscrowd": 0
                    })
                    
                except Exception as e:
                    continue
        
        # Sauvegarder l'image annotée
        annotated_img.save(annotated_path)
        
    except Exception as e:
        print(f"Erreur avec {img_name}: {str(e)}")
        continue

# Finalisation
driver.quit()
with open(COCO_FILE, 'w') as f:
    json.dump(coco_output, f, indent=2)

print("\n✅ Annotation terminée avec succès!")
print(f"Images annotées: {len(os.listdir(ANNOTATED_DIR))}/{TEST_LIMIT}")
print(f"Fichier COCO généré: {COCO_FILE}")

SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:14471
from session not created: This version of ChromeDriver only supports Chrome version 135
Current browser version is 134.0.6998.178
Stacktrace:
	GetHandleVerifier [0x00E55413+60691]
	GetHandleVerifier [0x00E55454+60756]
	(No symbol) [0x00C80693]
	(No symbol) [0x00CB9E1B]
	(No symbol) [0x00CB8E39]
	(No symbol) [0x00CAF5AF]
	(No symbol) [0x00CAF3E6]
	(No symbol) [0x00CF8DE1]
	(No symbol) [0x00CF87AA]
	(No symbol) [0x00CECE56]
	(No symbol) [0x00CBC5F3]
	(No symbol) [0x00CBD444]
	GetHandleVerifier [0x0109BBC3+2446531]
	GetHandleVerifier [0x01097198+2427544]
	GetHandleVerifier [0x010B2D5E+2541150]
	GetHandleVerifier [0x00E6C3C5+154821]
	GetHandleVerifier [0x00E72BED+181485]
	GetHandleVerifier [0x00E5CC48+91464]
	GetHandleVerifier [0x00E5CDF0+91888]
	GetHandleVerifier [0x00E47980+4736]
	BaseThreadInitThunk [0x77787BA9+25]
	RtlInitializeExceptionChain [0x77DBC0CB+107]
	RtlClearBits [0x77DBC04F+

mauvais toute la page est add

In [13]:
import os
import json
import time
from tqdm import tqdm
from PIL import Image, ImageDraw
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# Configuration
IMG_DIR = "dataset_images"
ANNOTATED_DIR = "dataset_images_annotated"
META_FILE = "dataset_metadata.json"
COCO_FILE = "annotations_adaptatives.json"
os.makedirs(ANNOTATED_DIR, exist_ok=True)
TEST_LIMIT=2
# Classes et couleurs
CLASSES = {
    "title": "h1, h2",
    "content": "main, article, [role=main]",
    "media": "img, video",
    "header": "header",
    "footer": "footer",
    "ads": "[class*='ad'], [id*='ad'], iframe[src*='ads']",
    "sidebar": "aside, [class*='sidebar']"
}

COLORS = {
    "title": "#FF0000",  # Rouge
    "content": "#0000FF",  # Bleu
    "media": "#00FF00",  # Vert
    "header": "#800080",  # Violet
    "footer": "#FFA500",  # Orange
    "ads": "#FFFF00",  # Jaune
    "sidebar": "#00FFFF"  # Cyan
}

# Initialisation COCO
coco = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": idx+1, "name": name, "supercategory": "web"} 
        for idx, name in enumerate(CLASSES.keys())
    ]
}

# Setup Chrome (taille arbitraire car nous allons tout recalculer)
options = uc.ChromeOptions()
options.add_argument("--force-device-scale-factor=1")
options.headless = True
driver = uc.Chrome(options=options)

# Chargement metadata
with open(META_FILE) as f:
    metadata = json.load(f)

def get_element_dimensions(driver, element):
    """Obtient les dimensions ABSOLUES d'un élément en JavaScript pur"""
    return driver.execute_script(
        "var rect = arguments[0].getBoundingClientRect();"
        "return {"
        "  x: rect.left + window.pageXOffset,"
        "  y: rect.top + window.pageYOffset,"
        "  width: rect.width,"
        "  height: rect.height"
        "};",
        element
    )

for img_idx, (img_name, meta) in enumerate(tqdm(list(metadata.items())[:TEST_LIMIT])):
    try:
        img_path = os.path.join(IMG_DIR, img_name)
        annotated_path = os.path.join(ANNOTATED_DIR, img_name)
        
        # Charger l'image et obtenir ses dimensions RÉELLES
        with Image.open(img_path) as img:
            img_width, img_height = img.size
            draw = ImageDraw.Draw(img)
            
            # Configurer le navigateur avec la taille de l'image
            driver.set_window_size(img_width, img_height)
            driver.get(meta["url"])
            time.sleep(3)
            
            # Scroll complet
            for _ in range(2):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0, 0)")
                time.sleep(0.5)
            
            # Enregistrement dans COCO
            coco["images"].append({
                "id": img_idx + 1,
                "file_name": img_name,
                "width": img_width,
                "height": img_height,
                "url": meta["url"]
            })
            
            # Annotation des éléments
            for class_id, (class_name, selector) in enumerate(CLASSES.items(), start=1):
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                
                for elem in elements:
                    try:
                        if not elem.is_displayed():
                            continue
                            
                        # Obtenir dimensions ABSOLUES
                        dim = get_element_dimensions(driver, elem)
                        x, y = int(dim['x']), int(dim['y'])
                        w, h = int(dim['width']), int(dim['height'])
                        
                        # Vérification des limites
                        if (w < 10 or h < 10 or 
                            x >= img_width or y >= img_height or
                            x + w <= 0 or y + h <= 0):
                            continue
                            
                        # Ajustement final
                        x = max(0, x)
                        y = max(0, y)
                        w = min(w, img_width - x)
                        h = min(h, img_height - y)
                        
                        # Dessin
                        draw.rectangle(
                            [x, y, x + w, y + h],
                            outline=COLORS[class_name],
                            width=3
                        )
                        draw.text(
                            (x + 5, y + 5),
                            class_name,
                            fill=COLORS[class_name]
                        )
                        
                        # Annotation COCO
                        coco["annotations"].append({
                            "id": len(coco["annotations"]) + 1,
                            "image_id": img_idx + 1,
                            "category_id": class_id,
                            "bbox": [x, y, w, h],
                            "area": w * h,
                            "iscrowd": 0
                        })
                        
                    except Exception as e:
                        continue
            
            # Sauvegarde
            img.save(annotated_path)
            
    except Exception as e:
        print(f"Erreur avec {img_name}: {str(e)}")
        continue

# Nettoyage
driver.quit()

# Sauvegarde COCO
with open(COCO_FILE, 'w') as f:
    json.dump(coco, f, indent=2)

print("\n✅ Annotation ADAPTATIVE terminée!")
print(f"• Images traitées: {len(os.listdir(ANNOTATED_DIR))}/{TEST_LIMIT}")
print(f"• Fichier COCO: {COCO_FILE}")

SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:14437
from session not created: This version of ChromeDriver only supports Chrome version 135
Current browser version is 134.0.6998.178
Stacktrace:
	GetHandleVerifier [0x00915413+60691]
	GetHandleVerifier [0x00915454+60756]
	(No symbol) [0x00740693]
	(No symbol) [0x00779E1B]
	(No symbol) [0x00778E39]
	(No symbol) [0x0076F5AF]
	(No symbol) [0x0076F3E6]
	(No symbol) [0x007B8DE1]
	(No symbol) [0x007B87AA]
	(No symbol) [0x007ACE56]
	(No symbol) [0x0077C5F3]
	(No symbol) [0x0077D444]
	GetHandleVerifier [0x00B5BBC3+2446531]
	GetHandleVerifier [0x00B57198+2427544]
	GetHandleVerifier [0x00B72D5E+2541150]
	GetHandleVerifier [0x0092C3C5+154821]
	GetHandleVerifier [0x00932BED+181485]
	GetHandleVerifier [0x0091CC48+91464]
	GetHandleVerifier [0x0091CDF0+91888]
	GetHandleVerifier [0x00907980+4736]
	BaseThreadInitThunk [0x77787BA9+25]
	RtlInitializeExceptionChain [0x77DBC0CB+107]
	RtlClearBits [0x77DBC04F+

In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install opencv-contrib-python selenium pillow numpy

   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/46.2 MB ? eta -:--:--
    --------------------------------------- 1.0/46.2 MB 5.2 MB/s eta 0:00:09
    --------------------------------------- 1.0/46.2 MB 5.2 MB/s eta 0:00:09
    --------------------------------------- 1.0/46.2 MB 5.2 MB/s eta 0:00:09
   - -------------------------------------- 1.3/46.2 MB 1.6 MB/s eta 0:00:28
   - -------------------------------------- 1.3/46.2 MB 1.6 MB/s eta 0:00:28
   - -------------------------------------- 1.6/46.2 MB 1.1 MB/s eta 0:00:43
   - -------------------------------------- 1.6/46.2 MB 1.1 MB/s eta 0:00:43
   - -------------------------------------- 1.6/46.2 MB 1.1 MB/s eta 0:00:43
   - -------------------------------------- 1.8/46.2 MB 871.4 kB/s eta 0:00:51
   - -------------------------------------- 1.8/46.2 MB 871.4 kB/s eta 0:00:51
   - -------------------------------------- 2.1/46.2 MB 829.6 kB/s eta 0:00:54
   - --

In [12]:
import os
import json
import time
import cv2
import numpy as np
from PIL import Image, ImageDraw
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ===== CONFIGURATION =====
TEST_IMAGE_COUNT = 2
IMG_DIR = "dataset_images"
ANNOTATED_DIR = "dataset_images_annotated_corrected"
META_FILE = "dataset_metadata.json"
os.makedirs(ANNOTATED_DIR, exist_ok=True)

# Classes à annoter
CLASSES = {
    "title": ("h1, h2", "#FF0000"),
    "content": ("main, article", "#0000FF"),
    "media": ("img, video", "#00FF00"),
}

# ===== FONCTIONS =====
def get_driver():
    """Initialisation robuste du navigateur"""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1280,800")  # Taille par défaut
    
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def process_image(driver, img_path, output_path, url):
    """Traitement complet d'une image"""
    try:
        # 1. Charger l'image
        img = Image.open(img_path)
        width, height = img.size
        draw = ImageDraw.Draw(img)
        
        # 2. Configurer le navigateur
        driver.set_window_size(width, height)
        driver.get(url)
        time.sleep(3)
        
        # 3. Capture pour alignement
        temp_path = "temp_sel.png"
        driver.save_screenshot(temp_path)
        sel_img = cv2.imread(temp_path)
        original_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        
        # 4. Alignement avec ORB
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(sel_img, None)
        kp2, des2 = orb.detectAndCompute(original_img, None)
        
        if des1 is not None and des2 is not None:
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(des1, des2)
            matches = sorted(matches, key=lambda x: x.distance)[:30]
            
            if len(matches) > 10:
                src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
                dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])
                M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            else:
                M = np.eye(3)
        else:
            M = np.eye(3)
        
        # 5. Annotation
        for class_name, (selector, color) in CLASSES.items():
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for elem in elements:
                try:
                    loc = elem.location
                    size = elem.size
                    
                    pts = np.float32([[
                        [loc['x'], loc['y']],
                        [loc['x']+size['width'], loc['y']],
                        [loc['x']+size['width'], loc['y']+size['height']],
                        [loc['x'], loc['y']+size['height']]
                    ]])
                    
                    transformed_pts = cv2.perspectiveTransform(pts, M)
                    draw.polygon(
                        [tuple(pt) for pt in transformed_pts[0]],
                        outline=color,
                        width=2
                    )
                    
                except Exception as e:
                    continue
        
        img.save(output_path)
        if os.path.exists(temp_path):
            os.remove(temp_path)
            
        return True
    
    except Exception as e:
        print(f"Erreur de traitement: {str(e)}")
        return False

# ===== EXÉCUTION =====
if __name__ == "__main__":
    print("🚀 Démarrage du processus...")
    
    try:
        # Charger les métadonnées avec vérification
        with open(META_FILE, "r") as f:
            metadata = json.load(f)
        
        # Filtrer les entrées valides
        valid_entries = [
            (k, v) for k, v in metadata.items() 
            if isinstance(v, dict) and 'url' in v and os.path.exists(os.path.join(IMG_DIR, k))
        ]
        
        if not valid_entries:
            raise ValueError("Aucune entrée valide dans les métadonnées")
        
        # Initialiser le driver
        driver = get_driver()
        success_count = 0
        
        # Traiter les images
        for img_name, info in valid_entries[:TEST_IMAGE_COUNT]:
            print(f"\n🔍 Traitement de {img_name}...")
            
            img_path = os.path.join(IMG_DIR, img_name)
            output_path = os.path.join(ANNOTATED_DIR, img_name)
            
            # Utiliser la taille de l'image si 'width' est absent
            img = Image.open(img_path)
            width, height = img.size
            
            if process_image(driver, img_path, output_path, info['url']):
                success_count += 1
                print(f"✅ Succès - {success_count}/{min(TEST_IMAGE_COUNT, len(valid_entries))}")
            else:
                print(f"❌ Échec sur {img_name}")
        
        print(f"\n🎉 Résultat final : {success_count}/{min(TEST_IMAGE_COUNT, len(valid_entries))} images annotées")
    
    except Exception as e:
        print(f"❌ Erreur globale : {str(e)}")
    
    finally:
        if 'driver' in locals():
            driver.quit()

🚀 Démarrage du processus...

🔍 Traitement de blog_OR_politics_OR_news_0_1743280479.jpg...
✅ Succès - 1/2

🔍 Traitement de blog_OR_politics_OR_news_1_1743280499.jpg...
✅ Succès - 2/2

🎉 Résultat final : 2/2 images annotées


bad

new test

In [1]:
import os
import json
import time
from PIL import Image, ImageDraw
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Configuration complète
CLASSES = {
    "header": {
        "keywords": ["header", "navbar", "head"],
        "color": "#FF0000"  # Rouge vif
    },
    "footer": {
        "keywords": ["footer", "bottom", "copyright"],
        "color": "#00FF00"  # Vert vif
    },
    "content": {
        "keywords": ["main", "content", "article"],
        "color": "#0000FF"  # Bleu
    },
    "media": {
        "keywords": ["img", "video", "picture"],
        "color": "#FF00FF"  # Magenta
    },
    "sidebar": {
        "keywords": ["sidebar", "aside", "panel"],
        "color": "#FFFF00"  # Jaune
    },
    "ads": {
        "keywords": ["ad", "banner", "advert"],
        "color": "#00FFFF"  # Cyan
    }
}

def load_metadata():
    """Charge les métadonnées depuis le fichier JSON"""
    with open("dataset_metadata.json") as f:
        return json.load(f)

def analyze_page(driver, url):
    """Analyse complète de la page"""
    driver.get(url)
    time.sleep(3)  # Attente critique pour le chargement
    
    # Récupération des dimensions du DOM
    dom_size = driver.execute_script("""
        return {
            width: Math.max(document.body.scrollWidth, document.documentElement.scrollWidth),
            height: Math.max(document.body.scrollHeight, document.documentElement.scrollHeight)
        }
    """)
    
    # Récupération de tous les éléments avec leur bounding box
    elements = driver.execute_script("""
        const elements = [];
        const all = document.querySelectorAll('*');
        
        all.forEach(el => {
            try {
                const rect = el.getBoundingClientRect();
                if (rect.width > 0 && rect.height > 0) {
                    elements.push({
                        tag: el.tagName,
                        classes: Array.from(el.classList),
                        id: el.id || null,
                        rect: {
                            x: rect.left,
                            y: rect.top,
                            width: rect.width,
                            height: rect.height
                        }
                    });
                }
            } catch(e) {}
        });
        
        return elements;
    """)
    
    return dom_size, elements

def match_element(element, classes):
    """Trouve la classe correspondante pour un élément"""
    for class_name, config in classes.items():
        # Vérification par tag
        if element['tag'].lower() in config['keywords']:
            return class_name, config['color']
        
        # Vérification par classe CSS
        for cls in element['classes']:
            if any(kw.lower() in cls.lower() for kw in config['keywords']):
                return class_name, config['color']
        
        # Vérification par ID
        if element['id'] and any(kw.lower() in element['id'].lower() for kw in config['keywords']):
            return class_name, config['color']
    
    return None, None

def annotate_image():
    # Initialisation
    metadata = load_metadata()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    # Création du dossier annoté
    os.makedirs("annotated", exist_ok=True)
    
    # Traitement de 2 images (comme demandé)
    test_images = list(metadata.items())[:2]
    
    for img_name, info in test_images:
        try:
            print(f"\nTraitement de {img_name}...")
            
            # Chemins des fichiers
            img_path = os.path.join("dataset_images", img_name)
            output_path = os.path.join("annotated", img_name)
            
            # Chargement de l'image
            img = Image.open(img_path)
            img_width, img_height = img.size
            
            # Analyse de la page
            dom_size, dom_elements = analyze_page(driver, info['url'])
            
            # Calcul des ratios de scaling
            scale_x = img_width / dom_size['width']
            scale_y = img_height / dom_size['height']
            
            # Préparation de l'annotation
            draw = ImageDraw.Draw(img)
            
            # Annotation des éléments
            for element in dom_elements:
                class_name, color = match_element(element, CLASSES)
                if class_name:
                    rect = element['rect']
                    
                    # Application du scaling
                    x1 = int(rect['x'] * scale_x)
                    y1 = int(rect['y'] * scale_y)
                    x2 = int((rect['x'] + rect['width']) * scale_x)
                    y2 = int((rect['y'] + rect['height']) * scale_y)
                    
                    # Vérification des limites
                    if x1 < img_width and y1 < img_height and x2 > 0 and y2 > 0:
                        # Rectangle avec bordure colorée
                        draw.rectangle(
                            [x1, y1, x2, y2],
                            outline=color,
                            width=3
                        )
                        # Texte d'étiquette
                        draw.text(
                            (x1 + 5, y1 + 5),
                            class_name,
                            fill=color
                        )
            
            # Sauvegarde
            img.save(output_path)
            print(f"✅ Annotation réussie -> {output_path}")
            
        except Exception as e:
            print(f"❌ Erreur sur {img_name}: {str(e)}")
    
    driver.quit()
    print("\nTraitement terminé !")

if __name__ == "__main__":
    annotate_image()


Traitement de blog_OR_politics_OR_news_0_1743280479.jpg...
✅ Annotation réussie -> annotated\blog_OR_politics_OR_news_0_1743280479.jpg

Traitement de blog_OR_politics_OR_news_1_1743280499.jpg...
✅ Annotation réussie -> annotated\blog_OR_politics_OR_news_1_1743280499.jpg

Traitement terminé !


le meilleur resulta obtenu meme si il n est pas preci . il reste a reduire fixer le nombre de header et footer a 1

In [7]:
import os
import json
import time
from PIL import Image, ImageDraw
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Configuration des classes avec couleurs et keywords
CLASS_CONFIG = {
    "header": {
        "keywords": ["header", "navbar", "topbar", "head", "navigation"],
        "color": "#FF0000",  # Rouge
        "full_width": True,
        "max_count": 1  # Un seul header
    },
    "footer": {
        "keywords": ["footer", "bottom", "copyright", "legal"],
        "color": "#00FF00",  # Vert
        "full_width": True,
        "max_count": 1  # Un seul footer
    },
    "content": {
        "keywords": ["main", "content", "article", "post", "body"],
        "color": "#0000FF"  # Bleu
    },
    "media": {
        "keywords": ["img", "video", "picture", "media", "gallery"],
        "color": "#FF00FF"  # Magenta
    },
    "sidebar": {
        "keywords": ["sidebar", "aside", "widget", "panel"],
        "color": "#FFFF00"  # Jaune
    },
    "ads": {
        "keywords": ["ad", "banner", "pub", "advert"],
        "color": "#00FFFF"  # Cyan
    }
}

def get_dom_elements(driver):
    """Récupère tous les éléments du DOM avec leurs propriétés"""
    return driver.execute_script("""
    const elements = [];
    const all = document.querySelectorAll('*');
    
    all.forEach(el => {
        try {
            const rect = el.getBoundingClientRect();
            if (rect.width > 5 && rect.height > 5) {  // Filtre les éléments trop petits
                elements.push({
                    tag: el.tagName.toLowerCase(),
                    classes: Array.from(el.classList),
                    id: el.id || null,
                    rect: {
                        x: rect.x,
                        y: rect.y,
                        width: rect.width,
                        height: rect.height
                    }
                });
            }
        } catch(e) {}
    });
    
    return elements;
    """)

def match_element(element, class_config):
    """Détermine la classe d'un élément selon les keywords"""
    element_str = f"{element['tag']} {' '.join(element['classes'])} {element['id']}".lower()
    
    for class_name, config in class_config.items():
        keywords = config['keywords']
        # Vérifie si un keyword est présent dans les classes, id ou tag
        if any(keyword.lower() in element_str for keyword in keywords):
            return class_name
    return None

def scale_coordinates(rect, img_size, dom_size):
    """Convertit les coordonnées du DOM vers l'image"""
    scale_x = img_size[0] / dom_size[0]
    scale_y = img_size[1] / dom_size[1]
    
    return {
        'x1': int(rect['x'] * scale_x),
        'y1': int(rect['y'] * scale_y),
        'x2': int((rect['x'] + rect['width']) * scale_x),
        'y2': int((rect['y'] + rect['height']) * scale_y)
    }

def annotate_image():
    # Initialisation
    with open("dataset_metadata.json") as f:
        metadata = json.load(f)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    os.makedirs("annotated", exist_ok=True)
    
    # Traitement de 2 images
    processed = 0
    for img_name, info in metadata.items():
        if processed >= 2:
            break
            
        try:
            print(f"\nTraitement de {img_name}...")
            
            # Chemins des fichiers
            img_path = os.path.join("dataset_images", img_name)
            output_path = os.path.join("annotated", img_name)
            
            # Chargement image
            img = Image.open(img_path)
            img_size = img.size
            
            # Analyse de la page
            driver.get(info['url'])
            time.sleep(3)  # Attente du chargement
            
            # Récupération des éléments et dimensions
            dom_elements = get_dom_elements(driver)
            dom_size = (
                driver.execute_script("return document.documentElement.scrollWidth"),
                driver.execute_script("return document.documentElement.scrollHeight")
            )
            
            # Annotation
            draw = ImageDraw.Draw(img)
            found_classes = {k: 0 for k in CLASS_CONFIG.keys()}
            
            for element in dom_elements:
                class_name = match_element(element, CLASS_CONFIG)
                
                if class_name:
                    config = CLASS_CONFIG[class_name]
                    
                    # Vérification du nombre max d'éléments
                    if config.get('max_count', float('inf')) <= found_classes[class_name]:
                        continue
                    
                    # Conversion des coordonnées
                    coords = scale_coordinates(element['rect'], img_size, dom_size)
                    
                    # Ajustement pour full-width
                    if config.get('full_width', False):
                        coords['x1'] = 0
                        coords['x2'] = img_size[0]
                    
                    # Dessin
                    draw.rectangle(
                        [coords['x1'], coords['y1'], coords['x2'], coords['y2']],
                        outline=config['color'],
                        width=3
                    )
                    draw.text(
                        (coords['x1'] + 5, coords['y1'] + 5),
                        class_name.upper(),
                        fill=config['color']
                    )
                    
                    found_classes[class_name] += 1
            
            img.save(output_path)
            processed += 1
            print(f"✅ Annotation réussie ({processed}/2)")
            
        except Exception as e:
            print(f"❌ Erreur: {str(e)}")
    
    driver.quit()
    print("\nTraitement terminé !")

if __name__ == "__main__":
    annotate_image()


Traitement de blog_OR_politics_OR_news_0_1743280479.jpg...
✅ Annotation réussie (1/2)

Traitement de blog_OR_politics_OR_news_1_1743280499.jpg...
✅ Annotation réussie (2/2)

Traitement terminé !


acceptable . probleme : mal detecter et beaucoup de content

grounded sam

In [2]:
# 🧰 Étape 1 : Clonage des repositories GitHub
!git clone https://github.com/UX-Decoder/Semantic-SAM.git
!git clone https://github.com/UX-Decoder/Segment-Everything-Everywhere-All-At-Once.git
!git clone https://github.com/IDEA-Research/OpenSeeD.git

# 📂 Étape 2 : Explorer les classes ou labels (si disponibles dans les fichiers)
import os
from pathlib import Path

repos = {
    "Semantic-SAM": "Semantic-SAM",
    "SEEM": "Segment-Everything-Everywhere-All-At-Once",
    "OpenSeeD": "OpenSeeD"
}

for name, repo_path in repos.items():
    print(f"\n📁 {name} — Possible fichiers de classes ou labels :")
    for root, dirs, files in os.walk(repo_path):
        for file in files:
            if any(keyword in file.lower() for keyword in ['class', 'label', 'category', 'vocab']):
                print("➡️", os.path.join(root, file))


Cloning into 'Semantic-SAM'...
Cloning into 'Segment-Everything-Everywhere-All-At-Once'...
Updating files:  11% (29/242)
Updating files:  12% (30/242)
Updating files:  13% (32/242)
Updating files:  14% (34/242)
Updating files:  15% (37/242)
Updating files:  16% (39/242)
Updating files:  17% (42/242)
Updating files:  18% (44/242)
Updating files:  19% (46/242)
Updating files:  20% (49/242)
Updating files:  21% (51/242)
Updating files:  22% (54/242)
Updating files:  23% (56/242)
Updating files:  24% (59/242)
Updating files:  25% (61/242)
Updating files:  26% (63/242)
Updating files:  27% (66/242)
Updating files:  28% (68/242)
Updating files:  29% (71/242)
Updating files:  30% (73/242)
Updating files:  31% (76/242)
Updating files:  32% (78/242)
Updating files:  33% (80/242)
Updating files:  34% (83/242)
Updating files:  35% (85/242)
Updating files:  36% (88/242)
Updating files:  37% (90/242)
Updating files:  38% (92/242)
Updating files:  39% (95/242)
Updating files:  40% (97/242)
Updating 


📁 Semantic-SAM — Possible fichiers de classes ou labels :

📁 SEEM — Possible fichiers de classes ou labels :
➡️ Segment-Everything-Everywhere-All-At-Once\datasets\evaluation\classification_evaluation.py

📁 OpenSeeD — Possible fichiers de classes ou labels :


Cloning into 'OpenSeeD'...
